# Spaceship Titanic 데이터셋
* PassengerId (승객 ID): 각 승객에 대한 고유 ID. gggg_pp 형식으로, gggg는 승객이 함께 여행 중인 그룹을 나타내며 pp는 그룹 내 번호입니다. 그룹은 종종 가족 구성원이지만 항상 그렇지는 않습니다.
* HomePlanet (출발 행성): 승객이 출발한 행성으로, 일반적으로 그들의 영구 거주지입니다.
* CryoSleep (동결 수면): 승객이 항해 기간 동안 동결 수면에 들 것인지 여부를 나타냅니다. 동결 수면 중인 승객은 자신의 객실에 제한됩니다.
* Cabin (객실): 승객이 머무는 객실 번호입니다. deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)일 수 있습니다.
* Destination (도착 행성): 승객이 하선할 행성입니다.
* Age (나이): 승객의 나이입니다.
* VIP (VIP 여부): 승객이 항해 중 특별한 VIP 서비스를 결제했는지 여부입니다.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck (부대시설 사용 금액): 승객이 Spaceship Titanic의 여러 고급 시설에서 청구한 금액입니다.
* Name (이름): 승객의 이름(성과 이름)


### 계획
* 일단 그냥 돌려보기
* 이상치 제거해보기
* 스케일링 다른거 해보기
* 오버샘플링기법해보기



In [ ]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler
import catboost
from catboost import Pool

# train 전처리

In [ ]:
titanic_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/train.csv')
titanic_train.drop(['Name'],axis=1, inplace=True)
# NA값 Unknown으로 대체
titanic_train['CryoSleep'] = titanic_train['CryoSleep'].fillna('Unknown')
# cost에 관련한 변수들은 중앙값으로 대체
# 각 cost에 관련된 변수들의 중앙값 계산
median_values = titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].median()
# 중앙값으로 대체
titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(median_values)
# NA값 Unknown으로 대체
titanic_train['HomePlanet'].fillna('Unknown', inplace=True)
# vip는 True, False로 이루어진 값임으로 결측치를 변경
titanic_train['VIP'] = titanic_train['VIP'].fillna('Unknown')
# Age가 NaN인 애들 평균으로 변경
age_mean = titanic_train['Age'].mean()
titanic_train['Age'] = titanic_train['Age'].fillna(age_mean)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터임으로 NaN값 삭제하고싶지만 예측에 중요할듯하니 살려둬
# titanic_train = titanic_train.dropna(subset=['Cabin'])
titanic_train['Cabin'] = titanic_train['Cabin'].fillna('Unknown/Unknown/Unknown')
# NA값 Unknown으로 대체
titanic_train['Destination'].fillna('Unknown', inplace = True)
# titanic_train['GroupNum'] = pd.to_numeric(titanic_train['PassengerId'].str.split('_').str[0])
titanic_train['GroupNumCnt'] = titanic_train['PassengerId'].str.split('_').str[0].map(titanic_train['PassengerId'].str.split('_').str[0].value_counts())
titanic_train = titanic_train.drop('PassengerId',axis=1)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터
# Cabin의 데이터 쪼개서 각각의 변수로 저장
titanic_train['deck'] = titanic_train['Cabin'].apply(lambda x : x.split('/')[0])
titanic_train['roomNum'] = pd.to_numeric(titanic_train['Cabin'].apply(lambda x : 0 if x.split('/')[1] == 'Unknown' else x.split('/')[1]))
titanic_train['side'] = titanic_train['Cabin'].apply(lambda x : x.split('/')[2])
titanic_train = titanic_train.drop('Cabin',axis=1)

titanic_train = pd.get_dummies(titanic_train,columns= titanic_train.select_dtypes('object').columns)

X_train, X_test, y_train, y_test = train_test_split(titanic_train.drop('Transported',axis=1), titanic_train['Transported'], test_size = 0.2, random_state = 2023)

In [ ]:
# 이상치
titanic_train[(titanic_train['ShoppingMall'] > 20000) | (titanic_train['FoodCourt'] > 20000) | (titanic_train['Spa'] > 20000) | (titanic_train['VRDeck'] > 20000)| (titanic_train['RoomService'] > 20000)]

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,GroupNumCnt,roomNum,HomePlanet_Earth,...,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T,deck_Unknown,side_P,side_S,side_Unknown
1213,31.0,0.0,27723.0,3.0,202.0,3146.0,True,2,55,0,...,0,0,0,0,0,0,0,0,1,0
1842,27.0,0.0,21066.0,0.0,2448.0,2293.0,True,1,75,0,...,1,0,0,0,0,0,0,0,1,0
2067,41.0,0.0,29813.0,0.0,1257.0,6.0,True,4,27,0,...,0,0,0,0,0,0,0,0,1,0
3198,33.0,0.0,27071.0,0.0,15.0,0.0,True,4,131,0,...,0,0,0,0,0,0,0,0,1,0
3538,33.0,90.0,26830.0,0.0,27.0,703.0,True,1,121,0,...,0,0,0,0,0,0,0,1,0,0
4311,31.0,0.0,6670.0,217.0,625.0,20336.0,False,4,144,0,...,1,0,0,0,0,0,0,1,0,0
5619,57.0,200.0,9.0,0.0,0.0,24133.0,False,4,230,0,...,0,0,0,0,0,0,0,0,1,0
5722,68.0,125.0,13437.0,0.0,22408.0,17.0,False,1,186,0,...,1,0,0,0,0,0,0,1,0,0
8415,17.0,0.0,366.0,23492.0,356.0,3.0,True,4,291,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        8693 non-null   float64
 1   RoomService                8693 non-null   float64
 2   FoodCourt                  8693 non-null   float64
 3   ShoppingMall               8693 non-null   float64
 4   Spa                        8693 non-null   float64
 5   VRDeck                     8693 non-null   float64
 6   Transported                8693 non-null   bool   
 7   GroupNumCnt                8693 non-null   int64  
 8   roomNum                    8693 non-null   int64  
 9   HomePlanet_Earth           8693 non-null   uint8  
 10  HomePlanet_Europa          8693 non-null   uint8  
 11  HomePlanet_Mars            8693 non-null   uint8  
 12  HomePlanet_Unknown         8693 non-null   uint8  
 13  CryoSleep_False            8693 non-null   uint8

# test 데이터 정제

In [ ]:
titanic_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/test.csv')
titanic_test.drop(['Name'],axis=1, inplace=True)
# NA값 Unknown으로 대체
titanic_test['CryoSleep'] = titanic_test['CryoSleep'].fillna('Unknown')
# cost에 관련한 변수들은 중앙값으로 대체
# 기존의 중앙값으로 대체
titanic_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = titanic_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(median_values)
# NA값 Unknown으로 대체
titanic_test['HomePlanet'].fillna('Unknown', inplace=True)
# vip는 True, False로 이루어진 값임으로 결측치를 변경
titanic_test['VIP'] = titanic_test['VIP'].fillna('Unknown')
# Age가 NaN인 애들 평균으로 변경
titanic_test['Age'] = titanic_test['Age'].fillna(age_mean)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터임으로 NaN값 삭제하고싶지만 예측에 중요할듯하니 살려둬
# titanic_train = titanic_train.dropna(subset=['Cabin'])
titanic_test['Cabin'] = titanic_test['Cabin'].fillna('Unknown/Unknown/Unknown')
# NA값 Unknown으로 대체
titanic_test['Destination'].fillna('Unknown', inplace = True)

In [ ]:
# titanic_train['GroupNum'] = pd.to_numeric(titanic_train['PassengerId'].str.split('_').str[0])
titanic_test['GroupNumCnt'] = titanic_test['PassengerId'].str.split('_').str[0].map(titanic_test['PassengerId'].str.split('_').str[0].value_counts())
titanic_test = titanic_test.drop('PassengerId',axis=1)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터
# Cabin의 데이터 쪼개서 각각의 변수로 저장
titanic_test['deck'] = titanic_test['Cabin'].apply(lambda x : x.split('/')[0])
titanic_test['roomNum'] = pd.to_numeric(titanic_test['Cabin'].apply(lambda x : 0 if x.split('/')[1] == 'Unknown' else x.split('/')[1]))
titanic_test['side'] = titanic_test['Cabin'].apply(lambda x : x.split('/')[2])
titanic_test = titanic_test.drop('Cabin',axis=1)

In [ ]:
titanic_test = pd.get_dummies(titanic_test,columns= titanic_test.select_dtypes('object').columns)
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        4277 non-null   float64
 1   RoomService                4277 non-null   float64
 2   FoodCourt                  4277 non-null   float64
 3   ShoppingMall               4277 non-null   float64
 4   Spa                        4277 non-null   float64
 5   VRDeck                     4277 non-null   float64
 6   GroupNumCnt                4277 non-null   int64  
 7   roomNum                    4277 non-null   int64  
 8   HomePlanet_Earth           4277 non-null   uint8  
 9   HomePlanet_Europa          4277 non-null   uint8  
 10  HomePlanet_Mars            4277 non-null   uint8  
 11  HomePlanet_Unknown         4277 non-null   uint8  
 12  CryoSleep_False            4277 non-null   uint8  
 13  CryoSleep_True             4277 non-null   uint8

# 데이터 분할

In [ ]:
def objective(trial):
    model = catboost.CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    # eval_set: 검증 세트(validation set) 튜플로 들어가기때문에 () 사용
    model.fit(X_train, y_train,eval_set=(X_test, y_test), early_stopping_rounds= 500, )
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=1500)

trial = study.best_trial
print("Best Trial AUC-ROC Score:", trial.value)
print("Best Params:", trial.params.items())

model = catboost.CatBoostClassifier(**trial.params, verbose=False)
model.fit(X_train, y_train)
y_train_pred_proba = model.predict_proba(X_train)[:, 1]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)

print("Train AUC-ROC Score:", train_auc_roc_score)
print("Test AUC-ROC Score:", test_auc_roc_score)


[I 2024-01-04 00:43:56,029] A new study created in memory with name: catboost
[I 2024-01-04 00:43:59,696] Trial 0 finished with value: 0.8987218886983037 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.8987218886983037.
[I 2024-01-04 00:44:05,515] Trial 1 finished with value: 0.8967187599263071 and parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 0 with value: 0.8987218886983037.
[I 2024-01-04 00:44:09,307] Trial 2 finished with value: 0.8567441977426256 and parameters: {'iterations': 226, 'learning_

Best Trial AUC-ROC Score: 0.905661039112297
Best Params: dict_items([('iterations', 919), ('learning_rate', 0.040356031006884514), ('depth', 4), ('l2_leaf_reg', 5.625637866533586e-05), ('bootstrap_type', 'Bayesian'), ('random_strength', 9.804387249766059e-08), ('bagging_temperature', 0.17859878258540401), ('od_type', 'Iter'), ('od_wait', 45)])
Train AUC-ROC Score: 0.9626104342893281
Test AUC-ROC Score: 0.905661039112297


In [ ]:
# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code
X_train, X_test, y_train, y_test = train_test_split(titanic_train.drop('Transported',axis=1), titanic_train['Transported'], test_size = 0.2, random_state = 2023)
# Convert training data to catboost.Pool
train_pool = Pool(X_train, label=y_train)

best_params = {'iterations': 919, 'learning_rate': 0.040356031006884514, 'depth': 4, 'l2_leaf_reg': 5.625637866533586e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 9.804387249766059e-08, 'bagging_temperature': 0.17859878258540401, 'od_type': 'Iter', 'od_wait': 45}

final_model = catboost.CatBoostClassifier(**best_params, verbose=False)
final_model.fit(train_pool)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

y_test_pred1 = final_model.predict(titanic_test)

# Get feature importance
feature_importance = final_model.get_feature_importance(data=train_pool,
                                                        type='LossFunctionChange',
                                                        prettified=False,
                                                        thread_count=-1,
                                                        verbose=False)

# Create a dictionary to store feature names and their importance
feature_importance_dict = {feature: importance for feature, importance in zip(X_train.columns, feature_importance)}

# Sort the dictionary by importance in ascending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print or use the sorted feature importance as needed
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")

Test AUC-ROC Score with Final Model: 0.905661039112297
Spa: 0.08563712186930506
VRDeck: 0.06559353513940569
RoomService: 0.03616532544572454
FoodCourt: 0.03093165069220255
roomNum: 0.028009655751565
Age: 0.018987750240738233
ShoppingMall: 0.01492159908785129
CryoSleep_True: 0.00864194167436988
HomePlanet_Earth: 0.007958298672789854
CryoSleep_False: 0.006690052011733949
deck_E: 0.005703420637959866
HomePlanet_Europa: 0.005593932054923534
side_S: 0.005515347475643639
deck_G: 0.0037768588421613347
side_P: 0.0036834832430017728
deck_C: 0.0033080878338480613
HomePlanet_Mars: 0.0029751371710413643
GroupNumCnt: 0.0026263178398716613
Destination_TRAPPIST-1e: 0.0022387978420064236
Destination_55 Cancri e: 0.0017623291301473998
deck_F: 0.0013303254448202573
Destination_PSO J318.5-22: 0.0012347947490953803
HomePlanet_Unknown: 0.001096698925454609
deck_A: 0.0009325300164672035
VIP_Unknown: 0.0005538091280415935
Destination_Unknown: 0.00046956084680876975
deck_B: 0.00046076479788470515
deck_D: 0.00

In [ ]:
titanic_test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/test.csv')
result_df = pd.DataFrame({'Transported': y_test_pred1, 'PassengerId':titanic_test1['PassengerId'] })

# Save to CSV without index
result_df.to_csv('y_test_pred_11try.csv', index=False)

# 트레인테스트비율조정


In [ ]:
titanic_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/train.csv')
titanic_train.drop(['Name'],axis=1, inplace=True)
# NA값 Unknown으로 대체
titanic_train['CryoSleep'] = titanic_train['CryoSleep'].fillna('Unknown')
# cost에 관련한 변수들은 중앙값으로 대체
# 각 cost에 관련된 변수들의 중앙값 계산
median_values = titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].median()
# 중앙값으로 대체
titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = titanic_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(median_values)
# NA값 Unknown으로 대체
titanic_train['HomePlanet'].fillna('Unknown', inplace=True)
# vip는 True, False로 이루어진 값임으로 결측치를 변경
titanic_train['VIP'] = titanic_train['VIP'].fillna('Unknown')
# Age가 NaN인 애들 평균으로 변경
age_mean = titanic_train['Age'].mean()
titanic_train['Age'] = titanic_train['Age'].fillna(age_mean)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터임으로 NaN값 삭제하고싶지만 예측에 중요할듯하니 살려둬
# titanic_train = titanic_train.dropna(subset=['Cabin'])
titanic_train['Cabin'] = titanic_train['Cabin'].fillna('Unknown/Unknown/Unknown')
# NA값 Unknown으로 대체
titanic_train['Destination'].fillna('Unknown', inplace = True)
# titanic_train['GroupNum'] = pd.to_numeric(titanic_train['PassengerId'].str.split('_').str[0])
titanic_train['GroupNumCnt'] = titanic_train['PassengerId'].str.split('_').str[0].map(titanic_train['PassengerId'].str.split('_').str[0].value_counts())
titanic_train = titanic_train.drop('PassengerId',axis=1)
# Cabin은 deck/num/side 형식으로, side는 P(왼쪽) 또는 S(오른쪽)으로 이루어진 데이터
# Cabin의 데이터 쪼개서 각각의 변수로 저장
titanic_train['deck'] = titanic_train['Cabin'].apply(lambda x : x.split('/')[0])
titanic_train['roomNum'] = pd.to_numeric(titanic_train['Cabin'].apply(lambda x : 0 if x.split('/')[1] == 'Unknown' else x.split('/')[1]))
titanic_train['side'] = titanic_train['Cabin'].apply(lambda x : x.split('/')[2])
titanic_train = titanic_train.drop('Cabin',axis=1)
titanic_train = pd.get_dummies(titanic_train,columns= titanic_train.select_dtypes('object').columns)

X_train, X_test, y_train, y_test = train_test_split(titanic_train.drop('Transported',axis=1), titanic_train['Transported'], test_size = 0.25, random_state = 42)

In [ ]:
def objective(trial):
    model = catboost.CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    # eval_set: 검증 세트(validation set) 튜플로 들어가기때문에 () 사용
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds= 500 )
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=1500)

trial = study.best_trial
print("Best Trial AUC-ROC Score:", trial.value)
print("Best Params:", trial.params.items())

model = catboost.CatBoostClassifier(**trial.params, verbose=False)
model.fit(X_train, y_train)
y_train_pred_proba = model.predict_proba(X_train)[:, 1]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)

print("Train AUC-ROC Score:", train_auc_roc_score)
print("Test AUC-ROC Score:", test_auc_roc_score)

[I 2024-01-04 02:42:09,490] A new study created in memory with name: catboost
[I 2024-01-04 02:42:12,668] Trial 0 finished with value: 0.8958096355277502 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.8958096355277502.
[I 2024-01-04 02:42:27,226] Trial 1 finished with value: 0.895129593142532 and parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 0 with value: 0.8958096355277502.
[I 2024-01-04 02:42:34,932] Trial 2 finished with value: 0.85647551001063 and parameters: {'iterations': 226, 'learning_rat

Best Trial AUC-ROC Score: 0.9040221946876291
Best Params: dict_items([('iterations', 823), ('learning_rate', 0.06047910839641756), ('depth', 4), ('l2_leaf_reg', 3.137289551040919), ('bootstrap_type', 'Bayesian'), ('random_strength', 9.992496746364536), ('bagging_temperature', 0.1607828365029389), ('od_type', 'Iter'), ('od_wait', 15)])
Train AUC-ROC Score: 0.9563951162492547
Test AUC-ROC Score: 0.9040221946876291


In [ ]:
# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code
# X_train, X_test, y_train, y_test = train_test_split(titanic_train.drop('Transported',axis=1), titanic_train['Transported'], test_size = 0.25, random_state = 2023)
# Convert training data to catboost.Pool
train_pool = Pool(X_train, label=y_train)

best_params = {'iterations': 823, 'learning_rate': 0.06047910839641756, 'depth': 4, 'l2_leaf_reg': 3.137289551040919, 'bootstrap_type': 'Bayesian', 'random_strength': 9.992496746364536, 'bagging_temperature': 0.1607828365029389, 'od_type': 'Iter', 'od_wait': 15}

final_model = catboost.CatBoostClassifier(**best_params, verbose=False)
final_model.fit(train_pool)

y_test_pred = final_model.predict(titanic_test)

# Evaluate on the train set
y_train_pred_proba = final_model.predict_proba(X_train)[:, 1]
train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC-ROC Score with Final Model:", train_auc_roc_score)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

# Get feature importance
feature_importance = final_model.get_feature_importance(data=train_pool,
                                                        type='LossFunctionChange',
                                                        prettified=False,
                                                        thread_count=-1,
                                                        verbose=False)

# Create a dictionary to store feature names and their importance
feature_importance_dict = {feature: importance for feature, importance in zip(X_train.columns, feature_importance)}

# Sort the dictionary by importance in ascending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print or use the sorted feature importance as needed
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")


Train AUC-ROC Score with Final Model: 0.955228985587326
Test AUC-ROC Score with Final Model: 0.9008130307265528
Spa: 0.06674541623099733
VRDeck: 0.05284481161920068
RoomService: 0.03346711880043973
roomNum: 0.026220540925518954
FoodCourt: 0.026201597651548003
ShoppingMall: 0.01597083007428257
Age: 0.015404874485812181
CryoSleep_True: 0.013020226239907229
HomePlanet_Earth: 0.00692813500611783
side_P: 0.005921098120979573
HomePlanet_Europa: 0.0058365271599549184
CryoSleep_False: 0.005410990341482846
deck_E: 0.004577681550132218
deck_G: 0.0040903112128575225
deck_C: 0.003565623761406622
side_S: 0.0030035034124097915
Destination_TRAPPIST-1e: 0.0024429582722516607
GroupNumCnt: 0.002372015337960942
HomePlanet_Mars: 0.002203448988018275
deck_F: 0.0016052967000951268
Destination_55 Cancri e: 0.0012093184910504373
Destination_PSO J318.5-22: 0.0009535819092450493
HomePlanet_Unknown: 0.0007980731811071395
deck_A: 0.0006466437830306426
deck_B: 0.0006037554532275502
VIP_Unknown: 0.00045960617028306

In [ ]:
titanic_test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/test.csv')
result_df = pd.DataFrame({'Transported': y_test_pred, 'PassengerId':titanic_test1['PassengerId'] })

# Save to CSV without index
result_df.to_csv('y_test_pred_14try.csv', index=False)

In [ ]:
# 변수 좀 삭제해보기
selected_columns = [col for col in titanic_train.columns if all(exclusion not in col for exclusion in ['VIP', 'deck','Transported'])]
X_train, X_test, y_train, y_test = train_test_split(titanic_train[selected_columns], titanic_train['Transported'], test_size = 0.2, random_state = 2023)
X_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupNumCnt,roomNum,HomePlanet_Earth,HomePlanet_Europa,...,CryoSleep_False,CryoSleep_True,CryoSleep_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,side_P,side_S,side_Unknown
4674,11.0,0.0,0.0,0.0,0.0,0.0,3,805,1,0,...,1,0,0,0,0,1,0,1,0,0
6702,40.0,0.0,0.0,0.0,0.0,0.0,1,1161,1,0,...,0,1,0,0,0,1,0,0,1,0
8548,36.0,794.0,371.0,0.0,228.0,1.0,2,1870,1,0,...,1,0,0,0,0,1,0,1,0,0
5194,26.0,0.0,1583.0,0.0,232.0,8040.0,1,187,0,1,...,1,0,0,0,0,1,0,1,0,0
1837,0.0,0.0,0.0,0.0,0.0,0.0,3,316,1,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,15.0,1042.0,0.0,36.0,0.0,0.0,1,1185,1,0,...,1,0,0,0,0,1,0,0,1,0
6049,34.0,0.0,618.0,0.0,0.0,10888.0,2,76,0,1,...,1,0,0,1,0,0,0,0,1,0
2743,29.0,0.0,0.0,0.0,0.0,0.0,4,469,1,0,...,0,1,0,1,0,0,0,0,1,0
5657,24.0,718.0,0.0,265.0,302.0,384.0,2,383,1,0,...,1,0,0,0,0,1,0,1,0,0


In [ ]:
def objective(trial):
    model = catboost.CatBoostClassifier(
        task_type="GPU",
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    # eval_set: 검증 세트(validation set) 튜플로 들어가기때문에 () 사용
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds= 500 )
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=1500)

trial = study.best_trial
print("Best Trial AUC-ROC Score:", trial.value)
print("Best Params:", trial.params.items())

model = catboost.CatBoostClassifier(**trial.params, verbose=False)
model.fit(X_train, y_train)
y_train_pred_proba = model.predict_proba(X_train)[:, 1]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)

print("Train AUC-ROC Score:", train_auc_roc_score)
print("Test AUC-ROC Score:", test_auc_roc_score)

[I 2024-01-04 07:58:58,287] A new study created in memory with name: catboost
[I 2024-01-04 07:59:01,398] Trial 0 finished with value: 0.893777264256824 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.893777264256824.
[W 2024-01-04 07:59:02,552] Trial 1 failed with parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_va

KeyboardInterrupt: ignored

In [ ]:
# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code

# Convert training data to catboost.Pool
train_pool = Pool(X_train, label=y_train)

best_params = dict(trial.params)

final_model = catboost.CatBoostClassifier(**best_params, verbose=False)
final_model.fit(train_pool)
y_test_pred = final_model.predict(titanic_test[selected_columns])
# Evaluate on the train set
y_train_pred_proba = final_model.predict_proba(X_train)[:, 1]
train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC-ROC Score with Final Model:", train_auc_roc_score)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

# Get feature importance
feature_importance = final_model.get_feature_importance(data=train_pool,
                                                        type='LossFunctionChange',
                                                        prettified=False,
                                                        thread_count=-1,
                                                        verbose=False)

# Create a dictionary to store feature names and their importance
feature_importance_dict = {feature: importance for feature, importance in zip(X_train.columns, feature_importance)}

# Sort the dictionary by importance in ascending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print or use the sorted feature importance as needed
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")

Train AUC-ROC Score with Final Model: 0.9294986553955525
Test AUC-ROC Score with Final Model: 0.8956328219299918
Spa: 0.059808127026423075
VRDeck: 0.05672992670291027
RoomService: 0.029830322963433742
FoodCourt: 0.02760499338961364
roomNum: 0.021447411418192508
ShoppingMall: 0.012534010051760636
HomePlanet_Europa: 0.01131301256313244
Age: 0.011131133368145363
HomePlanet_Earth: 0.010810162745437779
CryoSleep_True: 0.008761804905319415
CryoSleep_False: 0.0058893717793770395
side_P: 0.003971121559044166
side_S: 0.002924929027756984
HomePlanet_Mars: 0.0027864701430233008
GroupNumCnt: 0.0019705930966478213
Destination_55 Cancri e: 0.0018199004128870988
Destination_TRAPPIST-1e: 0.001497462495630708
HomePlanet_Unknown: 0.0007520322562265536
Destination_PSO J318.5-22: 0.0006357834236843662
Destination_Unknown: 0.00040246544463323763
CryoSleep_Unknown: 0.00021326372290458862
side_Unknown: 0.00017646064035431452


In [ ]:
titanic_test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/test.csv')
result_df = pd.DataFrame({'Transported': y_test_pred, 'PassengerId':titanic_test1['PassengerId'] })

# Save to CSV without index
result_df.to_csv('y_test_pred_13try.csv', index=False)

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code

# Create AdaBoost classifier
final_model = AdaBoostClassifier(n_estimators=50, learning_rate=1.0, random_state=42)

# Fit the model on the training data
final_model.fit(X_train, y_train)

# Evaluate on the train set
y_train_pred_proba = final_model.predict_proba(X_train)[:, 1]
train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC-ROC Score with Final Model:", train_auc_roc_score)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

# Generate predictions on the test set
y_test_pred = final_model.predict(X_test)

Train AUC-ROC Score with Final Model: 0.8936647462837162
Test AUC-ROC Score with Final Model: 0.8764876225568049


In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code

# Create CatBoost classifier
final_model = CatBoostClassifier(iterations=50, learning_rate=1.0, random_state=42, verbose=False)

# Fit the model on the training data
final_model.fit(X_train, y_train)

# Evaluate on the train set
y_train_pred_proba = final_model.predict_proba(X_train)[:, 1]
train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC-ROC Score with Final Model:", train_auc_roc_score)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

# Generate predictions on the test set
y_test_pred = final_model.predict(X_test)


Train AUC-ROC Score with Final Model: 0.9706313259294546
Test AUC-ROC Score with Final Model: 0.8724919530737141


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler

# Assuming X_train, y_train, X_test, y_test are defined somewhere before this code

def objective(trial):
    model = AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 1000),
        learning_rate=trial.suggest_float("learning_rate", 0.001, 1.0, log=True),
        algorithm=trial.suggest_categorical("algorithm", ["SAMME", "SAMME.R"]),
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred_proba)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="adaboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=750)

trial = study.best_trial
print("Best Trial AUC-ROC Score:", trial.value)
print("Best Params:", trial.params.items())

model = AdaBoostClassifier(**trial.params, random_state=42)
model.fit(X_train, y_train)
y_train_pred_proba = model.predict_proba(X_train)[:, 1]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)

print("Train AUC-ROC Score:", train_auc_roc_score)
print("Test AUC-ROC Score:", test_auc_roc_score)


[I 2024-01-04 10:38:38,434] A new study created in memory with name: adaboost
[I 2024-01-04 10:38:42,797] Trial 0 finished with value: 0.8795568367109249 and parameters: {'n_estimators': 446, 'learning_rate': 0.14486833887239342, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.8795568367109249.
[I 2024-01-04 10:38:44,383] Trial 1 finished with value: 0.7191232291468141 and parameters: {'n_estimators': 189, 'learning_rate': 0.0018924123965722254, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.8795568367109249.
[I 2024-01-04 10:38:49,126] Trial 2 finished with value: 0.8698872900916926 and parameters: {'n_estimators': 427, 'learning_rate': 0.04134759007834719, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.8795568367109249.
[I 2024-01-04 10:38:51,957] Trial 3 finished with value: 0.8784298699786122 and parameters: {'n_estimators': 244, 'learning_rate': 0.43087600478391286, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.8795568367109249.
[I 2024-01-04 10:38:

Best Trial AUC-ROC Score: 0.8803039567583593
Best Params: dict_items([('n_estimators', 614), ('learning_rate', 0.10171946403770894), ('algorithm', 'SAMME.R')])
Train AUC-ROC Score: 0.8962238415064929
Test AUC-ROC Score: 0.8803039567583593


In [ ]:
# Convert training data to catboost.Pool
train_pool = Pool(X_train, label=y_train)

# Best parameters from optimization/trials (replace this with your actual parameters)
best_params = dict(trial.params)

# Create AdaBoostClassifier with DecisionTreeClassifier as base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)  # You can adjust the max_depth as needed
final_model = AdaBoostClassifier(base_estimator=base_estimator, **best_params)

# Fit the model on the training data
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(titanic_test)

# Evaluate on the train set
y_train_pred_proba = final_model.predict_proba(X_train)[:, 1]
train_auc_roc_score = roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC-ROC Score with Final Model:", train_auc_roc_score)

# Evaluate on the test set
y_test_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_auc_roc_score = roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC-ROC Score with Final Model:", test_auc_roc_score)

# Get feature importance
feature_importance = final_model.feature_importances_

# Create a dictionary to store feature names and their importance
feature_importance_dict = {feature: importance for feature, importance in zip(X_train.columns, feature_importance)}

# Sort the dictionary by importance in ascending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print or use the sorted feature importance as needed
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Train AUC-ROC Score with Final Model: 0.8962238415064929
Test AUC-ROC Score with Final Model: 0.8803039567583593
roomNum: 0.1758957654723127
Spa: 0.12052117263843648
VRDeck: 0.12052117263843648
FoodCourt: 0.09446254071661238
ShoppingMall: 0.0781758957654723
RoomService: 0.0732899022801303
Age: 0.05537459283387622
deck_G: 0.050488599348534204
deck_C: 0.04234527687296417
side_P: 0.026058631921824105
deck_B: 0.024429967426710098
HomePlanet_Europa: 0.02280130293159609
HomePlanet_Earth: 0.017915309446254073
Destination_55 Cancri e: 0.017915309446254073
GroupNumCnt: 0.014657980456026058
CryoSleep_True: 0.013029315960912053
deck_E: 0.013029315960912053
side_S: 0.011400651465798045
CryoSleep_False: 0.008143322475570033
Destination_TRAPPIST-1e: 0.008143322475570033
Destination_Unknown: 0.004885993485342019
deck_A: 0.004885993485342019
deck_F: 0.0016286644951140066
HomePlanet_Mars: 0.0
HomePlanet_Unknown: 0.0
CryoSleep_Unknown: 0.0
Destination_PSO J318.5-22: 0.0
VIP_False: 0.0
VIP_True: 0.0
VIP_

In [ ]:
titanic_test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/8. 머신러닝과 딥러닝/240101 과제/space/test.csv')
result_df = pd.DataFrame({'Transported': y_test_pred, 'PassengerId':titanic_test1['PassengerId'] })

# Save to CSV without index
result_df.to_csv('y_test_pred_15try.csv', index=False)